檢查 AWS 權限


In [8]:
import os
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv

# 環境變數
load_dotenv()

def check_model_access():
    # 取得密鑰
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    region_name = os.getenv("AWS_REGION")

    # 確認環境變數
    if not all([aws_access_key_id, aws_secret_access_key, region_name]):
        print("錯誤：環境變數中未設定 AWS 憑證或區域。")
        return

    # 建立 AWS Bedrock 客戶端
    client = boto3.client(
        "bedrock-runtime",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name,
    )
    # 模型 ID
    model_id = "anthropic.claude-v2"  

    try:
        # 調用模型
        response = client.invoke_model(
            modelId=model_id,
            body=b'Test input to check access.'
        )
        print("成功：訪問模型已經被允許。")
    except ClientError as e:
        if e.response["Error"]["Code"] == "AccessDeniedException":
            print(f"Access Denied: {e.response['Error']['Message']}")
        else:
            print(f"Error: {e.response['Error']['Message']}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    check_model_access()


Access Denied: You don't have access to the model with the specified model ID.


檢查

In [7]:
import os
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

def check_model_access(model_id):
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    region_name = os.getenv("AWS_REGION")

    if not all([aws_access_key_id, aws_secret_access_key, region_name]):
        print("错误：环境变量中未设置 AWS 凭证或区域。")
        return

    client = boto3.client(
        "bedrock-runtime",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name,
    )

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=b'Test input to check access.'
        )
        print("成功：访问模型已经被允许。")
        print(response)
    except ClientError as e:
        if e.response["Error"]["Code"] == "AccessDeniedException":
            print(f"Access Denied: {e.response['Error']['Message']}")
        else:
            print(f"Error: {e.response['Error']['Message']}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    # 检查特定模型的访问权限
    model_id = "anthropic.claude-v2"  # 替换为你要检查的模型 ID
    check_model_access(model_id)


Access Denied: You don't have access to the model with the specified model ID.
